## Imports

In [20]:
import numpy as np
import pandas as pd
from rectools.models import PopularModel
from rectools.dataset import Dataset
from rectools import Columns
from rectools.metrics import Precision, Recall, MAP, MeanInvUserFreq, Serendipity, calc_metrics

## Data

In [2]:
interactions = pd.read_csv('../data/kion_train/interactions.csv')
users = pd.read_csv('../data/kion_train/users.csv')
items = pd.read_csv('../data/kion_train/items.csv')

In [3]:
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [4]:
max_date = interactions[Columns.Datetime].max().normalize()
max_date

Timestamp('2021-08-22 00:00:00')

## Train

In [14]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=None,
    item_features_df=None
)

In [25]:
popular_model = PopularModel()
popular_model.fit(dataset);

In [ ]:
pop = popular_model.recommend(
    dataset.user_id_map.external_ids[:1],
    dataset=dataset,
    k=30,
    filter_viewed=False)

In [26]:
popular_model_mean_weights = PopularModel(popularity="mean_weight")
popular_model_mean_weights.fit(dataset);

In [27]:
pop_mean_weights = popular_model_mean_weights.recommend(
    dataset.user_id_map.external_ids[:1],
    dataset=dataset,
    k=30,
    filter_viewed=False)

In [36]:
pop.head(10)

,user_id,item_id,score,rank
0,176549,10440,202457.0,1
1,176549,15297,193123.0,2
2,176549,9728,132865.0,3
3,176549,13865,122119.0,4
4,176549,4151,91167.0,5
5,176549,3734,74803.0,6
6,176549,2657,68581.0,7
7,176549,4880,55043.0,8
8,176549,142,45367.0,9
9,176549,6809,40372.0,10


In [38]:
pop_mean_weights.head(10)

,user_id,item_id,score,rank
0,176549,1714,635143.666667,1
1,176549,3642,442423.000000,2
2,176549,856,400153.000000,3
3,176549,3480,373410.666667,4
4,176549,16216,340005.500000,5
5,176549,13395,339193.000000,6
6,176549,11566,328638.566667,7
7,176549,1914,294688.000000,8
8,176549,2788,290355.500000,9
9,176549,3750,285721.000000,10


## Metrics

In [31]:
metrics = {
    "prec@5": Precision(k=5),
    "recall@5": Recall(k=5),
    "MAP@5": MAP(k=5),
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "MAP@20": MAP(k=20),
    "prec@20": Precision(k=20),
    "recall@20": Recall(k=20),
    "MAP@100": MAP(k=100),
    "prec@100": Precision(k=100),
    "recall@100": Recall(k=100),
    "MAP@100": MAP(k=100),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

In [32]:
catalog = train[Columns.Item].unique()
metric__pop = calc_metrics(metrics, pop, test, train, catalog)
metric_pop_mean_weights = calc_metrics(metrics, pop_mean_weights, test, train, catalog)

In [33]:
print(metric_values_pop)

{'prec@5': 0.0, 'recall@5': 0.0, 'prec@10': 0.0, 'recall@10': 0.0, 'prec@20': 0.0, 'recall@20': 0.0, 'prec@100': 0.0, 'recall@100': 0.0, 'MAP@5': 0.0, 'MAP@20': 0.0, 'MAP@100': 0.0, 'novelty': 3.4365596012517443, 'serendipity': 0.0}


In [34]:
print(metric_values_pop_mean_weight)

{'prec@5': 0.0, 'recall@5': 0.0, 'prec@10': 0.0, 'recall@10': 0.0, 'prec@20': 0.0, 'recall@20': 0.0, 'prec@100': 0.0, 'recall@100': 0.0, 'MAP@5': 0.0, 'MAP@20': 0.0, 'MAP@100': 0.0, 'novelty': 18.41768334978919, 'serendipity': 0.0}
